- CNN policy ?
- grid search for HP tuning (OK)
- Increasingly difficult Environment
- Positive reward for populating increasingly "deep" blending tanks ?
- RL for chem sched paper (https://arxiv.org/pdf/2203.00636)
- Masking (https://sb3-contrib.readthedocs.io/en/master/modules/ppo_mask.html, https://arxiv.org/pdf/2006.14171)
    - Adding binary decision variables ?g  
    - Requires discrete action space (only integer flows -> treated as categories ?)
    - masking: disable incoming flows (resp. outgoing flows) for tanks at UB inv limit (resp. LB inv. limit), disable selling/buying when available = 0
    - multiple envs with multiple agents ? (MARL, https://arxiv.org/pdf/2103.01955)
        - Predict successive pipelines ("source > blend" then "blend > blend" (as many as required) then "blend > demand")
        - Each agent has access to the whole state
        - Action mask is derived from the previous agent's actions (0 if inventory at bounds or incoming flow already reserved, else 1)
        - https://github.com/Rohan138/marl-baselines3/blob/main/marl_baselines3/independent_ppo.py
- Safe RL: (https://proceedings.mlr.press/v119/wachi20a/wachi20a.pdf)
    - "Unsafe state" ? > Do not enforce constraints strictly, instead opt for early episode termination to show which states are unsafe ? 
    - Implementations:
        - https://pypi.org/project/fast-safe-rl/#description (Policy optimizers)
        - https://github.com/PKU-Alignment/safety-gymnasium/tree/main/safety_gymnasium (environments; "cost" ?)


1. Try other learning rates/CNN policies
2. Implement Masking with single agent
3. Try other ways to tell the model what are illegal/unsafe states (safe RL)
4. Try multiple agents

-----------------------

- Masking: Discretization of action space is too slow/might not work -> Need to implement masking for continuous action space
- Recurrent policy makes the most sense ? (window of demand forecasts)
- https://www.reddit.com/r/reinforcementlearning/comments/17l5b47/invalid_action_masking_when_action_space_is/
    - Suggestion of autoregressive model for having constraints respected: one predicted action is input to a second model
    - Suggestion of editing the distribution in such a way that the constraint is respected
- https://www.sciencedirect.com/science/article/pii/S0098135420301599
    - Choice of ELU activation ?
    - Choice of NN size ?
    - "The feature engineering in the net inventory means the network does not have to learn these relationships itself, which did help speed training." ?
- Simplify the problem (remove tanks 5 to 8), find the optimal solution with Gurobi

- remove all constraints except in/out
- https://arxiv.org/pdf/1711.11157
- https://arxiv.org/pdf/2111.01564
- Softmax with large coef to produce action mask
- Graph convolution NN instead of RNN ?
    - https://pytorch-geometric.readthedocs.io/en/latest/
    - Graph rep. learning - William L Hamilton

- DDPG
- Softmax
- ~~Remove non-selling rewards~~
- MultiplexNet
- Why softmax doesn't work ? -> gradient doesn't compute properly

- Finalize adjustment of flows
- Add more difficulty (bigger env)

- Added penalty types logging
- Added regularization, no success
- Tried SDE briefly, no success
- Hard/Unable to reproduce results on best configs (3 and 12)

Next steps:
- Find non-trivial optimal solution
- Need different env versions to compare how flows are processed and the impact on performance
- Read SDE method & distribution 
- Generate random supply/demands and solutions with solver; train classic NN in a supervised fashion (loss function = reward function ? ; multi-period aspect ?)
- "Hire" another student ?

In [1]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
try:
    print(curr_dir)
except:
    curr_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    os.chdir(curr_dir)
    print(curr_dir)

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL


In [2]:
import json
import numpy as np
import torch as th
from stable_baselines3 import PPO, DDPG, SAC, TD3
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.callbacks import *
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecCheckNan

from envs import BlendEnv, flatten_and_track_mappings, reconstruct_dict
from models import *
from math import exp, log
import yaml

import warnings
warnings.filterwarnings("ignore")

In [3]:
with open("./configs/29.yaml", "r") as f:
    s = "".join(f.readlines())
    cfg = yaml.load(s, Loader=yaml.FullLoader)
    
layout = "simplest"

In [4]:
if cfg["custom_softmax"]:
    policytype = CustomMLP_ACP_simplest_softmax
elif cfg["policytype"] == "MLP":
    policytype = "MlpPolicy"
elif cfg["policytype"] == "MLPtanh":
    policytype = CustomMLP_ACP_simplest_tanh
    
optimizer_cls = eval(cfg["optimizer"])

if cfg["model"]["act_fn"] == "ReLU":
    act_cls = th.nn.ReLU
elif cfg["model"]["act_fn"] == "tanh":
    act_cls = th.nn.Tanh
elif cfg["model"]["act_fn"] == "sigmoid":
    act_cls = th.nn.Sigmoid

In [5]:
with open(f"./configs/json/connections_{layout}.json" ,"r") as f:
    connections_s = f.readline()
connections = json.loads(connections_s)

with open(f"./configs/json/action_sample_{layout}.json" ,"r") as f:
    action_sample_s = f.readline()
action_sample = json.loads(action_sample_s)
action_sample_flat, mapp = flatten_and_track_mappings(action_sample)

In [6]:
def get_sbp(connections):
    sources = list(connections["source_blend"].keys())
    
    b_list = list(connections["blend_blend"].keys())
    for b in connections["blend_blend"].keys():
        b_list += connections["blend_blend"][b]
    b_list += list(connections["blend_demand"].keys())
    blenders = list(set(b_list))
    
    p_list = []
    for p in connections["blend_demand"].keys():
        p_list += connections["blend_demand"][p]
    demands = list(set(p_list))
    
    return sources, blenders, demands
sources, blenders, demands = get_sbp(connections)

In [7]:
T = 6
if layout == "base":
    sigma = {"s1":{"q1": 0.06}, "s2":{"q1": 0.26}}
    sigma_ub = {"p1":{"q1": 0.16}, "p2":{"q1": 1}}
    sigma_lb = {"p1":{"q1": 0}, "p2":{"q1": 0}}
else:
    sigma = {s:{"q1": 0.06} for s in sources}
    sigma_ub = {d:{"q1": 0.16} for d in demands}
    sigma_lb = {d:{"q1": 0} for d in demands}
    
s_inv_lb = {s: 0 for s in sources}
s_inv_ub = {s: 999 for s in sources}
d_inv_lb = {d: 0 for d in demands}
d_inv_ub = {d: 999 for d in demands}
betaT_d = {d: 1 for d in demands} # Price of sold products
b_inv_ub = {j: 30 for j in blenders} 
b_inv_lb = {j: 0 for j in blenders}
betaT_s = {s: cfg["env"]["product_cost"]  for s in sources} # Cost of bought products

if cfg["env"]["uniform_data"]:
    tau0   = {s: [np.random.normal(15, 2) for _ in range(13)] for s in sources}
    delta0 = {d: [np.random.normal(15, 2) for _ in range(13)] for d in demands}
else:
    tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
    delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [8]:
env = BlendEnv(v = False, T = T,
               D = cfg["env"]["D"], Q = cfg["env"]["Q"], P = cfg["env"]["P"], B = cfg["env"]["B"], Z = cfg["env"]["Z"], M = cfg["env"]["M"],
               reg = cfg["env"]["reg"], reg_lambda = cfg["env"]["reg_lambda"],
               MAXFLOW = cfg["env"]["maxflow"], alpha = cfg["env"]["alpha"], 
               beta = cfg["env"]["beta"], connections = connections, 
               action_sample = action_sample, tau0 = tau0, delta0 = delta0, sigma = sigma,
               sigma_ub = sigma_ub, sigma_lb = sigma_lb,
               s_inv_lb = s_inv_lb, s_inv_ub = s_inv_ub,
               d_inv_lb = d_inv_lb, d_inv_ub = d_inv_ub,
               betaT_d = betaT_d, betaT_s = betaT_s,
               b_inv_ub = b_inv_ub,
               b_inv_lb = b_inv_lb)

In [9]:
env = Monitor(env)
env = DummyVecEnv([lambda: env])
env = VecNormalize(env, 
                   norm_obs=cfg["obs_normalizer"], 
                   norm_reward=cfg["reward_normalizer"])
# env = VecCheckNan(env, raise_exception=True)

In [10]:
policy_kwargs = dict(
    net_arch=[dict(pi = [cfg["model"]["arch_layersize"]] * cfg["model"]["arch_n"], 
                   vf = [cfg["model"]["arch_layersize"]] * cfg["model"]["arch_n"])],
    activation_fn = act_cls,
    log_std_init = cfg["model"]["log_std_init"]
)

In [11]:
print(policytype)

if optimizer_cls == PPO:
    kwa = dict(policy = policytype, 
                env = env,
                tensorboard_log = "./logs",
                clip_range = cfg["model"]["clip_range"],
                learning_rate = cfg["model"]["lr"],
                ent_coef = cfg["model"]["ent_coef"],
                use_sde = cfg["model"]["use_sde"],
                batch_size = cfg["model"]["batch_size"],
                policy_kwargs = policy_kwargs)
    
else:
    kwa = dict(policy = policytype, 
                env = env,
                tensorboard_log = "./logs",
                batch_size = cfg["model"]["batch_size"],
                learning_rate = cfg["model"]["lr"])

model = optimizer_cls(**kwa)

if cfg["starting_point"]:
    model.set_parameters(cfg["starting_point"])

MlpPolicy


If batch_size = 64 and n_steps = 2048, then 1 epoch = 2048/64 = 32 batches

In [15]:
import datetime

bin_ = f"{(cfg['id']//12)*12 +1}-{(cfg['id']//12 +1)*12 }"
entcoef = str(model.ent_coef) if type(model) == PPO else ""
cliprange = str(model.clip_range(0)) if type(model) == PPO else ""
model_name = f"models/simplest/{bin_}/{cfg['id']}/{cfg['id']}_{datetime.datetime.now().strftime('%m%d-%H%M')}"
model_name

'models/simplest/25-36/29/29_0918-1408'

In [16]:
class CustomLoggingCallbackPPO(BaseCallback):
    def __init__(self, schedule_timesteps, start_log_std=2, end_log_std=-1, std_control = cfg["clipped_std"]):
        super().__init__(verbose = 0)
        self.print_flag = False
        self.std_control = std_control
        
        self.start_log_std = start_log_std
        self.end_log_std = end_log_std
        self.schedule_timesteps = schedule_timesteps
        self.current_step = 0
        
        self.pen_M, self.pen_B, self.pen_P, self.pen_reg = [[]]*4
        
    def _on_rollout_end(self) -> None:
        
        # self.logger.record("train/std", th.exp(self.model.policy.log_std).mean().item())
        self.logger.record("penalties/in_out", sum(self.pen_M)/len(self.pen_M))
        self.logger.record("penalties/buysell_bounds", sum(self.pen_B)/len(self.pen_B))
        self.logger.record("penalties/tank_bounds", sum(self.pen_P)/len(self.pen_P))
        # self.logger.record("penalties/regterm", sum(self.pen_reg)/len(self.pen_reg))
        
        self.pen_M, self.pen_B, self.pen_P, self.pen_reg = [], [], [], []
        
        
    def _on_step(self) -> bool:
        # params = [k for k in self.model.policy.named_parameters()]
            
        # for j in range(len(params)):
        #     if any(th.isnan(params[j][1]).flatten()):
        #         print("NAN detected")
        #         print(params[j])
        #         return(False)
        
        
        log_std: th.Tensor = self.model.policy.log_std
        t = self.locals["infos"][0]['dict_state']['t']
        
        if self.locals["dones"][0]: # record info at each episode end
            self.pen_M.append(self.locals["infos"][0]["pen_tracker"]["M"])
            self.pen_B.append(self.locals["infos"][0]["pen_tracker"]["B"])
            self.pen_P.append(self.locals["infos"][0]["pen_tracker"]["P"])
        
        if self.num_timesteps%2048 < 6 and t == 1: # start printing
            self.print_flag = True
            
        if self.print_flag:
            print("\nt:", t)
            if np.isnan(self.locals['rewards'][0]) or np.isinf(self.locals['rewards'][0]):
                print(f"is invalid reward {self.locals['rewards'][0]}")
            for i in ['obs_tensor', 'actions', 'values', 'clipped_actions', 'new_obs', 'rewards']:
                if i in self.locals:
                    print(f"{i}: " + str(self.locals[i]))
            if t == 6:
                self.print_flag = False
                print(f"\n\nLog-Std at step {self.num_timesteps}: {log_std.detach().numpy()}")
                print("\n\n\n\n\n")
                
        if self.std_control:
            progress = self.current_step / self.schedule_timesteps
            new_log_std = self.start_log_std + progress * (self.end_log_std - self.start_log_std)
            self.model.policy.log_std.data.fill_(new_log_std)
            self.current_step += 1
                
        return True

In [17]:
class CustomLoggingCallbackDDPG(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.print_flag = False
        
        self.pen_M, self.pen_B, self.pen_P, self.pen_reg = [], [], [], []
        
    def _on_rollout_end(self) -> None: ...
        
    def _on_step(self) -> bool:
        t = self.locals["infos"][0]['dict_state']['t']
        
        if self.locals["dones"][0]: # record info at each episode end
            self.pen_M.append(self.locals["infos"][0]["pen_tracker"]["M"])
            self.pen_B.append(self.locals["infos"][0]["pen_tracker"]["B"])
            self.pen_P.append(self.locals["infos"][0]["pen_tracker"]["P"])
            self.pen_reg.append(self.locals["infos"][0]["pen_tracker"]["reg"])
            
            self.total_rewards.append(self.locals['rewards'][0])
            
        
        if self.num_timesteps%2048 < 6 and t == 1: # start printing
            self.print_flag = True
            
        if self.print_flag:
            print("\nt:", t)
            if np.isnan(self.locals['rewards'][0]) or np.isinf(self.locals['rewards'][0]):
                print(f"is invalid reward {self.locals['rewards'][0]}")
            for i in ['obs_tensor', 'actions', 'values', 'new_obs', 'rewards']:
                if i in self.locals:
                    print(f"{i}: " + str(self.locals[i]))
            if t == 6:
                self.print_flag = False
                # print(f"\nAvg rewards over the last 100 episodes:{sum(self.total_rewards[-100:])/100} ; last reward: {self.total_rewards[-1]}")
                
                self.logger.record('train/learning_rate', self.model.learning_rate)
                self.logger.record("penalties/in_out", sum(self.pen_M)/len(self.pen_M))
                self.logger.record("penalties/buysell_bounds", sum(self.pen_B)/len(self.pen_B))
                self.logger.record("penalties/tank_bounds", sum(self.pen_P)/len(self.pen_P))
                self.logger.record("penalties/regterm", sum(self.pen_reg)/len(self.pen_reg))
        
                self.pen_M, self.pen_B, self.pen_P, self.pen_reg = [], [], [], []   
                
                print("\n\n\n\n\n")
                
        return True

In [18]:
total_timesteps = 1e5
log_callback = CustomLoggingCallbackPPO(schedule_timesteps=total_timesteps) if optimizer_cls == PPO else CustomLoggingCallbackDDPG()
callback = CallbackList([log_callback])
model_name

'models/simplest/25-36/29/29_0918-1408'

- Higher batch size, no STD control (25)
- making the env progressively harder (26)
- harder env (no incentive) from scratch (27)
- higher penalties for contraints (28)

In [16]:
logpath = model_name[len("models/"):]
print(f"logging at {logpath}")
model.learn(total_timesteps = total_timesteps,
            progress_bar = False,
            tb_log_name = logpath,
            callback = callback,
            reset_num_timesteps = False)

logging at simplest/13-24/23/23_0916-1521

t: 1
obs_tensor: tensor([[ 0.0000,  0.0000,  0.0000,  0.0000, 14.7422, 14.5545, 14.6201,  0.0000,
         15.1697, 15.2425, 17.4497, 12.5866, 14.7327,  0.0000, 18.2978, 13.2086,
          0.0000]])
actions: [[ 0.02349485 -0.30684718  0.07184143 -0.1634581 ]]
values: tensor([[-1.0350]])
clipped_actions: [[0.02349485 0.         0.07184143 0.        ]]
new_obs: [[ 0.04834658  0.02349485  0.          0.06       14.620074    0.
  15.169688   15.242549   17.449656   12.586577   14.732715    0.
  18.297802   13.208551   11.476386    0.          1.        ]]
rewards: [-2.814742]

t: 2
obs_tensor: tensor([[ 0.0483,  0.0235,  0.0000,  0.0600, 14.6201,  0.0000, 15.1697, 15.2425,
         17.4497, 12.5866, 14.7327,  0.0000, 18.2978, 13.2086, 11.4764,  0.0000,
          1.0000]])
actions: [[ 2.4307098 -7.8670273 -8.713527   3.757355 ]]
values: tensor([[-0.7191]])
clipped_actions: [[2.4307098 0.        0.        3.757355 ]]
new_obs: [[ 0.          0.071841

KeyboardInterrupt: 

In [229]:
import re

def save_next_file(directory, model_name):
    base_pattern = re.compile(model_name + r"_(\d+)\.zip")
    
    files = os.listdir(directory)
    
    max_number = 0
    for file in files:
        match = base_pattern.match(file)
        if match:
            number = int(match.group(1))
            max_number = max(max_number, number)
    
    # Generate the next filename
    next_file_number = max_number + 1
    next_file_name = f"{model_name}_{next_file_number}"
    next_file_path = os.path.join(directory, next_file_name)
    
    model.save(next_file_path)
    
save_next_file(os.path.dirname(model_name), os.path.basename(model_name) )

# Testing

In [22]:
model = PPO.load("models\\simplest\\25-36\\29\\29_0918-1503.zip")

In [23]:
# M,Q,P,B,Z,D = 10, 0, 5, 5, 1, 0
M, Q, P, B, Z, D  = cfg["env"]["M"], cfg["env"]["Q"], cfg["env"]["P"], cfg["env"]["B"], cfg["env"]["Z"], 0
# M,Q,P,B,Z,D = 0, 0, 0, 0, 1, 0

In [24]:
if cfg["env"]["uniform_data"]:
    tau0   = {s: [np.random.binomial(1, 0.7) * np.random.normal(15, 2) for _ in range(13)] for s in sources}
    delta0 = {d: [np.random.binomial(1, 0.7) * np.random.normal(15, 2) for _ in range(13)] for d in demands}
else:
    tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
    delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [25]:
env = BlendEnv(v = True, 
               D = cfg["env"]["D"], 
               Q = cfg["env"]["Q"], 
               P = cfg["env"]["P"], 
               B = cfg["env"]["B"], 
               Z = cfg["env"]["Z"], 
               M = cfg["env"]["M"],
               reg = cfg["env"]["reg"],
               reg_lambda = cfg["env"]["reg_lambda"],
               MAXFLOW = cfg["env"]["maxflow"],
               alpha = cfg["env"]["alpha"],
               beta = cfg["env"]["beta"],
               connections = connections, 
               action_sample = action_sample,
               tau0 = tau0,delta0 = delta0,
               sigma = sigma,
               sigma_ub = sigma_ub, sigma_lb = sigma_lb,
               s_inv_lb = s_inv_lb, s_inv_ub = s_inv_ub,
               d_inv_lb = d_inv_lb, d_inv_ub = d_inv_ub,
               betaT_d = betaT_d, betaT_s = betaT_s,
               b_inv_ub = b_inv_ub,
               b_inv_lb = b_inv_lb)
env = Monitor(env)

In [26]:
with th.autograd.set_detect_anomaly(True):
    obs = env.reset()
    obs, obs_dict = obs
    for k in range(env.T):
        action, _ = model.predict(obs, deterministic=False)
        print("\n\n",env.pen_tracker)
        print(action)
        print("\n\n   ",reconstruct_dict(action, env.mapping_act))
        obs, reward, done, term, _ = env.step(action)
        print(obs)
        dobs = reconstruct_dict(obs, env.mapping_obs)
        print("\n    >>     ",dobs["sources"], dobs["blenders"], dobs["demands"])
        print("   " ,reward)
        



 {'M': 0, 'B': 0, 'P': 0, 'Q': 0, 'reg': 0}
[ 0.        0.       11.364664  0.      ]


    {'source_blend': {'s1': {'j1': 0.0}}, 'blend_demand': {'j1': {'p1': 0.0}}, 'tau': {'s1': 11.364664}, 'delta': {'p1': 0.0}}
[PEN] t1; s1:			bought too much (more than supply)
Increased reward by 0 through tank population in s1
j1: inv: 0, in_flow_sources: 0.0, in_flow_blend: 0, out_flow_blend: 0, out_flow_demands: 0.0
Increased reward by 0 through tank population in j1
Increased reward by 0 through tank population in p1
[ 0.        0.        0.        0.       14.644503 15.507391 17.358292
 14.621485 12.706727 17.684172  0.       16.69194  12.181685 13.330943
 12.561095 13.936094  1.      ]

    >>      {'s1': 0.0} {'j1': 0.0} {'p1': 0.0}
    -100.0


 {'M': 0, 'B': -100, 'P': 0, 'Q': 0, 'reg': 0}
[ 0.        0.       10.162529  0.      ]


    {'source_blend': {'s1': {'j1': 0.0}}, 'blend_demand': {'j1': {'p1': 0.0}}, 'tau': {'s1': 10.162529}, 'delta': {'p1': 0.0}}
Increased reward by 10.162528

In [33]:
th.Tensor(action)

tensor([0.0149, 0.0122, 0.0000, 0.0000])

In [253]:
# 0 (only once per episode)
episode_rewards = []
obs = env.reset()
obs, obs_dict = obs

In [262]:
# 1 Get first action
print(env.t)
action, _ = model.predict(obs, deterministic=True)

2


In [263]:
print(env.t)
d = reconstruct_dict(obs, env.mapping_obs)
print(d["sources"])
print(d["blenders"])
print(d["demands"])
print(d["properties"])

2
{'s1': 17.46205}
{'j1': 0.0}
{'p1': 0.0}
{'j1': {'q1': 0.0}}


In [264]:
# 2 Visualize action
print(env.t)
reconstruct_dict(action, env.mapping_act)

2


{'source_blend': {'s1': {'j1': 0.0}},
 'blend_demand': {'j1': {'p1': 30.307917}},
 'tau': {'s1': 8.731916},
 'delta': {'p1': 17.08481}}

In [265]:
# 3
# Step once: get 2nd action
print(env.t)
obs, reward, done, term, _ = env.step(action)

2


In [207]:
# 4 Visualize new state
print(env.t)
d = reconstruct_dict(obs, env.mapping_obs)
print(d["sources"])
print(d["blenders"])
print(d["demands"])
print(d["properties"])

3
{'s1': 26.193966}
{'j1': 0.0}
{'p1': 0.0}
{'j1': {'q1': 0.0}}


In [2]:
import tensorboard as tb

In [3]:
experiment_id = "c1KCv3X3QvGwaXfgX1c4tg"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df

ValueError: Error [from server]: 
****************************************************************
****************************************************************
****************************************************************

ERROR: TensorBoard.dev has been shut down.

This command is no longer operational and will be removed.

See the FAQ at https://tensorboard.dev.

****************************************************************
****************************************************************
****************************************************************
